In [2]:
#importing necessary libraries
import time
import matplotlib.pylab as plt
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import threading
import queue

In [1]:
class LaneDetection:
    #creating an constructor and initializing queue and model
    def __init__(self):
        self.que = queue.Queue()
        self.model = load_model(r"Models\unet_lane_97.h5")
    
    #function to extract region of interest
    def region_of_interest(self, img, mask):
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image
    


    #function to detect contours and draw them on image    
    def draw_contours(self, img1, img2):
        img2 = cv2.resize(img2, (256,256))
        contours, h = cv2.findContours(img1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            area = cv2.contourArea(c)
            if area > 4.5: #filtering contours having area < 4.5
                cv2.drawContours(img2, [c], 0, (0,255,0), -1)

        return img2
    
    def predict(self, image):
    
        #expanding image dimension (256,256,1) ---> (1,256,256,1) to be fed into model
        ip_image = np.expand_dims(image, axis = 0)

        #predicting image mask and rescaling it back by 255
        prediction = ((self.model.predict(ip_image)[0,:,:,0] > 0.5).astype(np.uint8))*255

        self.que.put(prediction)

    
    #defining a function to process every frame
    def process(self, image, frame):


        thread  = threading.Thread(target = self.predict, args = [image]) 
        thread.start()

        #blurring image to deal with noise
#         blur_image  = cv2.GaussianBlur(image, (7, 7), -1)

        #detecting edges using canny detection
        canny_image = cv2.Canny(image, 150, 300)
        prediction = self.que.get()

        #cropping image to predicted mask
        cropped_image = self.region_of_interest(canny_image, prediction)

        #drawing contours on cropped image
        line_image = self.draw_contours(cropped_image, frame)

        return line_image
    
        
    def stream_vid(self, cap):

        _, frame = cap.read()

        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.resize(gray_image, (256,256))

        img = self.process(gray_image, frame)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        return img